# 1. Calculate total earnings (sum of transactions above 0) for each user from Switzerland by year as a pivot table.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, year, sum

path_to = "input_csv_files/"

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Spark CSV Analysis") \
    .getOrCreate()

# Consolidate read options for simplicity
def read_spark_csv(path):
    return spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", ";") \
        .csv(path)

# Read CSV files
transactionsDF = read_spark_csv(path_to + "transactions.csv")
countryAbbrDF = read_spark_csv(path_to + "country_abbreviation.csv")
accountsDF = read_spark_csv(path_to + "accounts.csv")

# Join transactionsDF and accountsDF with only required columns and perform filter operation
filteredDF = transactionsDF.filter(col("amount") > 0) \
    .select("id", "amount", "transaction_date", "country") \
    .join(accountsDF.select("id", "first_name", "last_name"), "id")

# Join with country abbreviation dataframe and filter for Swiss users
swissEarningsDF = filteredDF \
    .join(countryAbbrDF, filteredDF["country"] == countryAbbrDF["abbreviation"]) \
    .filter(col("country_full_name") == "Switzerland")

# Calculate yearly earnings and pivot on 'year'
pivotDF = swissEarningsDF.groupBy(
    concat(col("first_name"), lit(" "), col("last_name")).alias("full_name"),
    year(col("transaction_date")).alias("year")) \
    .agg(sum("amount").alias("earnings")) \
    .groupBy("full_name").pivot("year").agg(sum("earnings"))

pivotDF.show()


+------------------+-------+------------------+-------+--------+-------+-------+------------------+--------+-----------------+-------+-------+
|         full_name|   2011|              2012|   2013|    2014|   2015|   2016|              2017|    2018|             2019|   2020|   2021|
+------------------+-------+------------------+-------+--------+-------+-------+------------------+--------+-----------------+-------+-------+
|    Natalie Nelson|   null|              null|   null|    null|1664.57| 7067.5|           7446.06|14378.04|             null|   null|   null|
|     Lenny Spencer|2050.35|              null|   null|    null|   null| 509.08|16116.580000000002|    null|          8693.52| 923.65|8797.15|
|      Lucia Watson|   null|           2173.26|   null|10047.25|   null|   null|              null|    null|          4500.61|   null|   null|
|    Adelaide Dixon|   null|              null|   null|    null|   null|   null|           2176.15|    null|             null|   null|   null|